In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy.stats import boxcox
from plotnine import *
from plotnine import ggplot, aes, geom_line
from pmdarima.arima import auto_arima
from statsmodels.tsa.arima.model import ARIMA
from tqdm.notebook import tqdm

In [2]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

## Pre-process Data

In [3]:
# Load Data
calendar = pd.read_csv("Data/calendar_afcs2020.csv")
sales = pd.read_csv("Data/sales_train_evaluation_afcs2020.csv")
train = pd.read_csv("Data/sales_train_validation_afcs2020.csv")
sample_submission = pd.read_csv("Data/sample_submission_afcs2020.csv")
price = pd.read_csv("Data/sell_prices_afcs2020.csv")

In [4]:
# Convert days into dates
calendar['date'] = pd.to_datetime(calendar['date'])
datetime = calendar[["date","d"]].copy()

In [5]:
# price["id"] = price["item_id"] + "_" + price["store_id"] + "_validation"

In [6]:
# # Visualization
# # plt.figure(figsize=(12,12))
# plt.plot(ts_train["ds"],ts_train["y"],label="Train")
# plt.plot(ts_test["ds"],ts_test["y"],label="Test")
# plt.plot(ts_test["ds"],pred,label="Forecast")
# plt.title("Time Series Forecasting")
# plt.legend()

## Forecast function

In [7]:
def forecast(h, MA=False, season_naive=False, auto_ARIMA=False):
    """Forecast based on historical sales of items"""
    
    # Do nothing if no forecast is selected
    if not MA and not season_naive and not auto_arima_f:
        print ("No forecast has been made")
        return 0
    
    # Set parameters
    output = {}

    # Forecast sales per product item
#     for i in range(1, len(train) + 1):
    for i in tqdm(range(1, len(train) + 1)):
        fc = []
        
        # Create train data per item and merge with datetime
        item_train = train.iloc[(i-1):i].iloc[:,1:].T.reset_index()
        item_train = pd.merge(left=item_train,right=datetime,left_on="index",right_on="d",how="left")
        item_train.drop(["d","index"],axis=1,inplace=True)
        item_train = item_train.rename({"date":"ds",item_train.columns[0]:"y"},axis=1)
        item_train = item_train[item_train.columns[::-1]]
        
        # Create test data per item and merge with datetime
        item_sales = sales.iloc[(i-1):i][sales.columns[-(h+1):]].iloc[:,1:].T.reset_index()
        item_sales = pd.merge(left=item_sales,right=datetime,left_on="index",right_on="d",how="left")
        item_sales.drop(["d","index"],axis=1,inplace=True)
        item_sales = item_sales.rename({"date":"ds",item_sales.columns[0]:"y"},axis=1)

        # Create time series of train and test data
        ts_train = item_train.iloc[-int(h/2*8):,:].copy()
        ts_test = item_sales[item_sales.columns[::-1]]

        # Forecast using Moving Average
        if MA:
            fc_str = "MA"
            data = ts_train["y"].tolist()
            pred = moving_average(data, h)[1:]
        
        if season_naive:
            fc_str = "season_naive"
            data = ts_train.set_index("ds")["y"]
            pred = s_naive(data, h)
            
        if auto_ARIMA:
            fc_str = "auto_arima"
            data = ts_train["y"].tolist()
            pred = auto_arima_f(data, h)
        
        # Create output
        fc.append(train.id[(i-1)])
        fc.extend(pred)
        output[(i-1)] = fc

    # Convert output to dataframe
    output_df = pd.DataFrame.from_dict(output, orient='index', columns=sample_submission.columns)
    output_df.to_csv('Output/output_{}.csv'.format(fc_str), index=False)
    
    return output_df

## Forecast Models

#### Moving Average Forecast (0.90348)

https://machinelearningmastery.com/time-series-forecasting-methods-in-python-cheat-sheet/

In [8]:
# Moving Average (MA)
def moving_average(train, h):
    
    # fit model
    model = ARIMA(train, order=(0, 0, 1))
    model_fit = model.fit()
    
    # make prediction
    start_index = len(train) + 1
    end_index = start_index + h
    fc = model_fit.predict(start=start_index, end=end_index)
    
    return fc

In [9]:
forecast(28, MA=True)

/usr/local/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
/usr/local/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
/usr/local/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
/usr/local/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
/usr/local/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
/usr/local/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
/usr/local/lib/python3.8/site-packages/s

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_2_001_CA_3_validation,0.115786,0.115786,0.115786,0.115786,0.115786,0.115786,0.115786,0.115786,0.115786,0.115786,0.115786,0.115786,0.115786,0.115786,0.115786,0.115786,0.115786,0.115786,0.115786,0.115786,0.115786,0.115786,0.115786,0.115786,0.115786,0.115786,0.115786,0.115786
1,HOBBIES_2_002_CA_3_validation,0.178493,0.178493,0.178493,0.178493,0.178493,0.178493,0.178493,0.178493,0.178493,0.178493,0.178493,0.178493,0.178493,0.178493,0.178493,0.178493,0.178493,0.178493,0.178493,0.178493,0.178493,0.178493,0.178493,0.178493,0.178493,0.178493,0.178493,0.178493
2,HOBBIES_2_003_CA_3_validation,0.670597,0.670597,0.670597,0.670597,0.670597,0.670597,0.670597,0.670597,0.670597,0.670597,0.670597,0.670597,0.670597,0.670597,0.670597,0.670597,0.670597,0.670597,0.670597,0.670597,0.670597,0.670597,0.670597,0.670597,0.670597,0.670597,0.670597,0.670597
3,HOBBIES_2_004_CA_3_validation,0.257982,0.257982,0.257982,0.257982,0.257982,0.257982,0.257982,0.257982,0.257982,0.257982,0.257982,0.257982,0.257982,0.257982,0.257982,0.257982,0.257982,0.257982,0.257982,0.257982,0.257982,0.257982,0.257982,0.257982,0.257982,0.257982,0.257982,0.257982
4,HOBBIES_2_005_CA_3_validation,0.089517,0.089517,0.089517,0.089517,0.089517,0.089517,0.089517,0.089517,0.089517,0.089517,0.089517,0.089517,0.089517,0.089517,0.089517,0.089517,0.089517,0.089517,0.089517,0.089517,0.089517,0.089517,0.089517,0.089517,0.089517,0.089517,0.089517,0.089517
5,HOBBIES_2_006_CA_3_validation,0.135097,0.135097,0.135097,0.135097,0.135097,0.135097,0.135097,0.135097,0.135097,0.135097,0.135097,0.135097,0.135097,0.135097,0.135097,0.135097,0.135097,0.135097,0.135097,0.135097,0.135097,0.135097,0.135097,0.135097,0.135097,0.135097,0.135097,0.135097
6,HOBBIES_2_007_CA_3_validation,0.580096,0.580096,0.580096,0.580096,0.580096,0.580096,0.580096,0.580096,0.580096,0.580096,0.580096,0.580096,0.580096,0.580096,0.580096,0.580096,0.580096,0.580096,0.580096,0.580096,0.580096,0.580096,0.580096,0.580096,0.580096,0.580096,0.580096,0.580096
7,HOBBIES_2_008_CA_3_validation,0.152115,0.152115,0.152115,0.152115,0.152115,0.152115,0.152115,0.152115,0.152115,0.152115,0.152115,0.152115,0.152115,0.152115,0.152115,0.152115,0.152115,0.152115,0.152115,0.152115,0.152115,0.152115,0.152115,0.152115,0.152115,0.152115,0.152115,0.152115
8,HOBBIES_2_009_CA_3_validation,-0.000005,-0.000005,-0.000005,-0.000005,-0.000005,-0.000005,-0.000005,-0.000005,-0.000005,-0.000005,-0.000005,-0.000005,-0.000005,-0.000005,-0.000005,-0.000005,-0.000005,-0.000005,-0.000005,-0.000005,-0.000005,-0.000005,-0.000005,-0.000005,-0.000005,-0.000005,-0.000005,-0.000005
9,HOBBIES_2_010_CA_3_validation,0.115371,0.115371,0.115371,0.115371,0.115371,0.115371,0.115371,0.115371,0.115371,0.115371,0.115371,0.115371,0.115371,0.115371,0.115371,0.115371,0.115371,0.115371,0.115371,0.115371,0.115371,0.115371,0.115371,0.115371,0.115371,0.115371,0.115371,0.115371


#### Seasonal Naive (Score: 1.15463)

https://pawarbi.github.io/blog/forecasting/r/python/rpy2/altair/fbprophet/ensemble_forecast/uncertainty/simulation/2020/04/21/timeseries-part2.html#Importing-libraries

In [ ]:
def pysnaive(train_series,seasonal_periods,forecast_horizon):    
    '''
    Python implementation of Seasonal Naive Forecast. 
    This should work similar to https://otexts.com/fpp2/simple-methods.html
    Returns two arrays
     > fitted: Values fitted to the training dataset
     > fcast: seasonal naive forecast
    
    Author: Sandeep Pawar
    
    Date: Apr 9, 2020
    
    Ver: 1.0
    
    train_series: Pandas Series
        Training Series to be used for forecasting. This should be a valid Pandas Series. 
        Length of the Training set should be greater than or equal to number of seasonal periods
        
    Seasonal_periods: int
        No of seasonal periods
        Yearly=1
        Quarterly=4
        Monthly=12
        Weekly=52
        

    Forecast_horizon: int
        Number of values to forecast into the future
    
    e.g. 
    fitted_values = pysnaive(train,12,12)[0]
    fcast_values = pysnaive(train,12,12)[1]
    '''
    
    if len(train_series)>= seasonal_periods: #checking if there are enough observations in the training data
        
        last_season=train_series.iloc[-seasonal_periods:]
        
        reps=np.int(np.ceil(forecast_horizon/seasonal_periods))
        
        fcarray=np.tile(last_season,reps)
        
        fcast=pd.Series(fcarray[:forecast_horizon])
        
        fitted = train_series.shift(seasonal_periods)
        
    else:
        fcast=print("Length of the trainining set must be greater than number of seasonal periods") 
    
    return fitted, fcast


def s_naive(data, h):
    """Call the Season Naive function"""
    
    # Fitted values
    py_snaive_fit = pysnaive(data, 
                         seasonal_periods=7,
                         forecast_horizon=h)[0]

    # Forecast 
    py_snaive = pysnaive(data, 
                         seasonal_periods=7,
                         forecast_horizon=h)[1]

    #Residuals
    py_snaive_resid = (data - py_snaive_fit).dropna()
    
    # Predict Season Naive
    pred = py_snaive.values
    
    return pred

In [ ]:
forecast(28, season_naive=True)

#### Auto Arima (Score: )

https://towardsdatascience.com/time-series-forecasting-using-auto-arima-in-python-bb83e49210cd

In [ ]:
def auto_arima_f(data, h):
    arima_model =  auto_arima(data,start_p=0, d=1, start_q=0, 
                              max_p=5, max_d=5, max_q=5, start_P=0, 
                              D=1, start_Q=0, max_P=5, max_D=5,
                              max_Q=5, m=12, seasonal=True, 
                              error_action='warn',trace = True,
                              supress_warnings=True,stepwise = True,
                              random_state=20,n_fits = 50)
    
    pred = arima_model.predict(n_periods = h)
    
    return pred

In [ ]:
forecast(28, auto_ARIMA=True)